In [246]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("train.csv")
df = df.drop(["PassengerId","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
    
df['honor'] = np.array(honor)
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
y = df["Survived"].as_matrix()

In [247]:
df = df.fillna(value = {'Embarked': 1})
df = df.fillna(value = {"Age":-1})

In [248]:
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Age","honor"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [249]:
df.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,FamilySize,Age,honor
0,0,3,"Braund, Mr. Owen Harris",0,1,0,A/5 21171,7.2500,0.0,2,22,1
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,PC 17599,71.2833,1.0,2,38,1
2,1,3,"Heikkinen, Miss. Laina",1,0,0,STON/O2. 3101282,7.9250,0.0,1,26,2
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,113803,53.1000,0.0,2,35,1
4,0,3,"Allen, Mr. William Henry",0,0,0,373450,8.0500,0.0,1,35,1


In [250]:
df = df.drop(["Survived","Name","SibSp","Parch","Ticket"], axis = 1)

In [251]:
df.head()

,Pclass,Sex,Fare,Embarked,FamilySize,Age,honor
0,3,0,7.2500,0.0,2,22,1
1,1,1,71.2833,1.0,2,38,1
2,3,1,7.9250,0.0,1,26,2
3,1,1,53.1000,0.0,2,35,1
4,3,0,8.0500,0.0,1,35,1


In [252]:
df.count()

Pclass        891
Sex           891
Fare          891
Embarked      891
FamilySize    891
Age           891
honor         891
dtype: int64

In [253]:
X = df.as_matrix()

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [255]:
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(X_train, y_train)
print('Train score: {}'.format(forest.score(X_train, y_train)))
print('Test score: {}'.format(forest.score(X_test, y_test)))

Train score: 0.8787425149700598
Test score: 0.8340807174887892


In [256]:
from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
gbct.fit(X_train, y_train)
print('train accuracy: %.3f' % gbct.score(X_train, y_train))
print('test  accuracy: %.3f' % gbct.score(X_test, y_test))

train accuracy: 0.907
test  accuracy: 0.861


In [260]:
import xgboost as xgb
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.877
test  accuracy: 0.848


In [269]:
from sklearn.model_selection import StratifiedKFold
kf= StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(gbct, X, y, cv=kf)
scores.mean()

0.83057376319558895

In [278]:
df2 = pd.read_csv("test.csv")
df2 = df2.drop(["PassengerId","Cabin"], axis = 1)
df2 = df2.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

honor1 = []
for i in df2["Name"]:
    if "Master" in i :
        honor1.append(0)
    elif "Mr" in i :
        honor1.append(1)
    elif "Miss" in i :
        honor1.append(2)
    elif "Mrs" in i :
        honor1.append(3)
    else:
        honor1.append("4")
    
df2['honor'] = np.array(honor1)
df2["FamilySize"] = df2["SibSp"] + df2["Parch"] + 1

df2 = df2.fillna(value = {'Embarked': 1})
df2 = df2.fillna(value = {"Age":-1})

b = df2.loc[:,["Age","honor"]]
b =np.array(b)
for i in b:
    if i[0] == -1:
        i[0] =df2[df2["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(b, columns=["Age", "honor"])
df2 = df2.drop(["Age","honor"], axis = 1)
df2 = pd.concat([df2, frame], axis=1)
df2 = df2.drop(["Name","Ticket","SibSp","Parch"],axis=1)
df2 = df2.fillna(df2.mean())

X1 = df2.as_matrix()

In [279]:
df2.count()

Pclass        418
Sex           418
Fare          418
Embarked      418
FamilySize    418
Age           418
honor         418
dtype: int64

In [280]:
y1 = gbct.predict(X1)
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred8.csv") 